# Tutorial 7: Potential field data inversion

In this tutorial we demonstrate how to invert gravity and magnetic data with open-source software.
The inversion routine uses [SimPEG](https://simpeg.xyz/) has the core algorithm. We will invert field and tensor data simulated over the synthetic Flin Flon deposit.

## User Notes

### Data Panel
 - **Survey Type:** Select one of *Magnetic* or *Gravity*
 - **Object:** Select a Point, Curve, Surface of Grid2D object present in your project geoh5 file
 - **Channels:** Select one or multiple data channels corresponding to the field and/or tensor data associated to the same *Survey Type* (e.g. TMI, bxx, byy, bzz for magnetic tensor data)
#### Component Panel
For each *Data Channel* selected above specify:
   - **Data type**
   - **Uncertainties** (default to a floor value = 5th percentile) 
 
### Spatial information
 - **Topography** 
  - [Option 1] Object: Select an object from the source geoh5 file and channel
  - [Option 2] Drape Height: Specify a drape offset from the data locations
 - **Sensor Height**
  - [Option 1] Channel: Use a specific data channel for the receiver heights or vertices [Default]
  - [Option 2] Drape Height: Specify a drape offset above topography. Requires topography.
  
### Data selection (window and downample)
Interactive data selection from the source object.

### Inversion options
Parameters controling the inversion routine (on-going)

In [1]:
# Run this cell only on first use to change SimPEG branch!
try:
    from SimPEG.Directives import VectorInversion
except ImportError as err:
    !pip install git+https://github.com/simpeg/simpeg.git@MagDev_DF_Dask --upgrade  

^C


In [1]:
from shutil import copyfile
from functions.widgets.selection import object_data_selection_widget
from functions.widgets.inversion import pf_inversion_widget

# Lets create a working copy
# my_h5file = r"assets/FlinFlon.geoh5"
my_h5file = r"C:\Users\dominiquef\Dropbox\Projects\Synthetic\Nut_Cracker\Nutcracker.geoh5"
new_file = copyfile(my_h5file, my_h5file[:-6] + "_pf_v2.geoh5")

# Start the potential fields widget
pf_inversion_widget(new_file, resolution=100, inducing_field="58500, 78, 7.5")

Congratulation
----

By completeing this demo, you have inverted potential field data over windowed regions of the Flin Flon deposit. You are invited to try the same process on your own data.  